In [2]:
import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

musings_path = 'sam_musings/'
musings = os.listdir(musings_path)

# Import Dataset
print('Loading musings...')
df = pd.DataFrame({'content': [open(musings_path + musing).read() for musing in musings][0:10]})

# Convert to list
data = df.content.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        # deacc=True removes punctuations

data_words = list(sent_to_words(data))

# Build the bigram and trigram models
print('Making bigrams...')
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

print('Making trigrams...')
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
print('Modding n-grams...')
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
print('Destroying stop words...')
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
print('Forming bigrams...')
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
print('Building spacy NLP parser...')
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
print('Lemmatizing bigrams...')
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
print('Building dictionary...')
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
print('Making corpus...')
corpus = [id2word.doc2bow(text) for text in texts]

topic_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 200]
lda_models = [] * len(topic_sizes)
coherence_model_ldas = [] * len(topic_sizes)
coherence_lda = np.zeros(len(topic_sizes))

# Build LDA model
print('Generating LDA Model...')
#for index, topics in enumerate(topic_sizes):
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=30, 
                                                   random_state=100,
                                                   update_every=1,
                                                   chunksize=100,
                                                   passes=10,
                                                   alpha='auto',
                                                   per_word_topics=True)
    
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Generate mallet topic
mallet_path = '../mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=32, limit=40, step=6)

# Show graph
limit=40; start=32; step=6;
x = range(start, limit, step)

# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

# Select the model and print the topics
optimal_model = model_list[np.argmax(coherence_values)]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)


Loading musings...
Making bigrams...
Making trigrams...
Modding n-grams...
Destroying stop words...
Forming bigrams...
Building spacy NLP parser...
Lemmatizing bigrams...
Building dictionary...
Making corpus...
Generating LDA Model...
('\nPerplexity: ', -7.101449534735451)
('\nCoherence Score: ', 0.3545563928835792)
('Num Topics =', 32, ' has Coherence Value of', 0.4853)
('Num Topics =', 38, ' has Coherence Value of', 0.4753)
[(5,
  u'0.125*"feed" + 0.111*"rss" + 0.069*"title" + 0.056*"http_www" + 0.049*"file" + 0.035*"gmt" + 0.035*"content" + 0.035*"reader" + 0.028*"cgi" + 0.028*"cs"'),
 (7,
  u'0.207*"grinnell" + 0.089*"variety" + 0.074*"cs" + 0.074*"problem" + 0.052*"technology" + 0.044*"plan" + 0.030*"break" + 0.030*"tool" + 0.022*"encourage" + 0.022*"build"'),
 (11,
  u'0.197*"major" + 0.091*"thing" + 0.061*"add" + 0.053*"curriculum" + 0.053*"question" + 0.045*"feel" + 0.045*"topic" + 0.038*"graduate" + 0.030*"issue" + 0.023*"low"'),
 (18,
  u'0.188*"teach" + 0.076*"design" + 0.04

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,24,0.2415,"faculty, decision, committee, governance, admi...","On Tuesday at lunch, I'm part of a small group..."
1,1,5,0.2927,"feed, rss, title, http_www, file, gmt, content...",A variety of people have asked that I set up a...
2,2,30,0.5365,"post, message, reply, original, list, quote, m...","In my copious spare time, I serve as one of th..."
3,3,11,0.1800,"major, thing, add, curriculum, question, feel,...",For as long as I've been a faculty member at G...
4,4,8,0.2474,"classic, artist, response, late, refer, act, n...","On Friday, I was reading the Scarlet and Black..."
5,5,29,0.1845,"online, goa, week, approach, canvas, face, reg...","This past semester, I've had my first experien..."
6,6,13,0.1468,"computer, session, program, talk, computer_sci...",Research evidence suggests that Grinnell secon...
7,7,22,0.1643,"bad, academic, explore, fun, nighter, reality,...",I've been making some bad choices recently. Th...
8,8,15,0.2656,"term, line, goal, kind, happen, request, bit, ...",I rant a lot about the administration. I there...
9,9,19,0.2986,"people, professional, fortunate, discipline, s...",I just returned from the 2018 SIGCSE Technical...


In [3]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,5,0.2927,"feed, rss, title, http_www, file, gmt, content...",A variety of people have asked that I set up a...
1,8,0.2474,"classic, artist, response, late, refer, act, n...","On Friday, I was reading the Scarlet and Black..."
2,11,0.1800,"major, thing, add, curriculum, question, feel,...",For as long as I've been a faculty member at G...
3,13,0.1468,"computer, session, program, talk, computer_sci...",Research evidence suggests that Grinnell secon...
4,15,0.2656,"term, line, goal, kind, happen, request, bit, ...",I rant a lot about the administration. I there...


In [10]:
sent_topics_sorteddf_mallet['Text'][1]

'On Friday, I was reading the Scarlet and Black, Grinnell\'s weekly newspaper [1]. In it was a short interview with the Overcoats [2], a band who are playing Grinnell on November 16. The first question is "What kind of music did you both grow up listening to?" Hana Elion\'s response starts "I liked the classics like [...]". Now, that\'s a phrase that many pop and rock musicians say. In the 60\'s, the classics typically referred to artists like Elvis, Howlin\' Wolf, Hank Williams, Junior Parker and such. By the 70\'s, the classics were artists like The Beatles, The Stones, The Who, Motown, Otis, James Brown, Dylan, Aretha and perhaps even the Velvets. Moving forward a decade or two, one would probably add artists like Springsteen, Fleetwood Mac, Prince, and Michael Jackson. I\'m not sure that anyone ever referred to the late 1970\'s punk and new wave acts as "classics". However, by this time, it seems that Patti Smith, the Ramones, Hsker D, the Jam, the Beat, and other such acts are cla

In [13]:
def getTopicForQuery (question):
    temp = question.lower()

    words = re.findall(r'\w+', temp, flags = re.UNICODE | re.LOCALE)

    important_words = []
    important_words = filter(lambda x: x not in stoplist, words)

    dictionary = corpora.Dictionary.load('questions.dict')

    ques_vec = []
    ques_vec = dictionary.doc2bow(important_words)

    topic_vec = []
    topic_vec = lda[ques_vec]

    word_count_array = numpy.empty((len(topic_vec), 2), dtype = numpy.object)
    for i in range(len(topic_vec)):
        word_count_array[i, 0] = topic_vec[i][0]
        word_count_array[i, 1] = topic_vec[i][1]

    idx = numpy.argsort(word_count_array[:, 1])
    idx = idx[::-1]
    word_count_array = word_count_array[idx]

    final = []
    final = lda.print_topic(word_count_array[0, 0], 1)

    question_topic = final.split('*') ## as format is like "probability * topic"

    return question_topic[1]

In [14]:
getTopicForQuery('Hi hecker.')

NameError: global name 'stoplist' is not defined